# Install Package

In [ ]:
!pip install transformers
!pip install tensorflow_hub

# Import Module

In [ ]:
# 기본 패키지
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

# 정규화 패키지
import re

# 모델링 패키지
import tensorflow as tf
import transformers
import tensorflow_hub as hub
from transformers import GPT2Tokenizer, TFGPT2Model, AutoTokenizer
from keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Input, Dropout, Lambda
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# NLP 패키지
from konlpy.tag import Okt, Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고무시
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [ ]:
df = pd.read_csv('Petition_data_advised.csv')
df.head()

,content,label
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 AI...,1
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...,1
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...,1
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을...",1
4,"12년간 당한 학교폭력에 관한 청원 저는 OO초등학교 2003년 입학, OO초등학교...",1


# Train, Test Split

In [ ]:
train_data = pd.concat([df[df['label'] == 1][:80], df[df['label'] == 0][:400]])
test_data = pd.concat([df[df['label'] == 1][80:], df[df['label'] == 0][400:]])

# Data Preprocessing

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [ ]:
max_seq_len = 128
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='', eos_token='', pad_token='')

X_train, y_train = convert_examples_to_features(train_data['content'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data['content'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 59/59 [00:00<00:00, 498.86it/s]


# Modeling

In [ ]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  
  earlystopping = EarlyStopping(monitor="val_loss", patience = 5)

  checkpoint = ModelCheckpoint('GPT_kor.h5',
                              save_best_only=True,
                              save_weights_only=True,
                              monitor='val_loss',
                              mode='min',
                              verbose=False)

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
history = model.fit(X_train, y_train, epochs=50, callbacks=[checkpoint, earlystopping], batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
15/15 [==============================] - 63s 702ms/step - loss: 6.0683 - accuracy: 0.6729 - val_loss: 0.6004 - val_accuracy: 0.7119
Epoch 2/50
15/15 [==============================] - 2s 117ms/step - loss: 0.9985 - accuracy: 0.7021 - val_loss: 0.8324 - val_accuracy: 0.7119
Epoch 3/50
15/15 [==============================] - 2s 116ms/step - loss: 0.6776 - accuracy: 0.7292 - val_loss: 0.6154 - val_accuracy: 0.7119
Epoch 4/50
15/15 [==============================] - 2s 148ms/step - loss: 0.5818 - accuracy: 0.7833 - val_loss: 0.7055 - val_accuracy: 0.7119
Epoch 5/50
15/15 [==============================] - 2s 114ms/step - loss: 0.5532 - accuracy: 0.8042 - val_loss: 0.6692 - val_accuracy: 0.7119
Epoch 6/50
15/15 [==============================] - 2s 118ms/step - loss: 0.6073 - accuracy: 0.8125 - val_loss: 0.6264 - val_accuracy: 0.7119


In [ ]:
# 저장된 checkpoint 로드 후 정확도 측정
model.load_weights('GPT_kor.h5')
model.evaluate(X_test, y_test)

2/2 [==============================] - 1s 33ms/step - loss: 0.6004 - accuracy: 0.7119


[0.600389838218689, 0.7118644118309021]

In [ ]:
# 예측
model.predict(X_test)[0]

2/2 [==============================] - 7s 3s/step


array([0.3450919], dtype=float32)